## Install Required Packages

In [21]:
!pip install --upgrade pip
!pip install -q openai
!pip install pdfplumber
!pip install tqdm


## Import Required Packages

In [7]:
import openai
import random
import pdfplumber
import tqdm

### Enter Your GPT Api Key

In [8]:
openai.api_key = ""

In [9]:
messages = [
    {"role": "system", "content": "You are a kind helpful assistant."},
]

## Function for extracting text from Files

In [22]:
def extract_text_from_pdf(pdf_file_path):
    text = ""
    with pdfplumber.open(pdf_file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Read the PDF file
pdf_file_path = "chapter-2.pdf"
pdf_text = extract_text_from_pdf(pdf_file_path)

# Split the PDF text into smaller chunks
text_chunks = [pdf_text[i:i+4090] for i in range(0, len(pdf_text), 4090)]

## Function for Summarize text which we get from previous function

In [11]:
def get_all_abs_text(text):
    #import pdb;pdb.set_trace()
    chat_history = [{"role": "system", "content": "You are a helpful assistant."}]
    abstracted_text = ''
    for i, chunk in tqdm(text,desc ="Summary"): 
        chat_history.append({"role": "user", "content": "Summarize the following text:"})
        chat_history.append({"role": "assistant", "content": chunk})
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=chat_history,
            max_tokens=300
        )
        summary = response['choices'][0]['message']['content']
        abstracted_text += summary
        chat_history = [{"role": "system", "content": "You are a helpful assistant."}]
    return abstracted_text


In [12]:
abstracted_text = get_all_abs_text(text_chunks)

Creating summary for Page No.1
Creating summary for Page No.2
Creating summary for Page No.3
Creating summary for Page No.4
Creating summary for Page No.5
Creating summary for Page No.6
Creating summary for Page No.7
Creating summary for Page No.8
Creating summary for Page No.9


## Function for To Create MCQ with multiple Ans

In [14]:

# Define a function to generate an MCQ with multiple answers
def generate_mcq_with_multiple_answers(input_text, num_choices=4, num_correct_answers=2):
    question = ""
    correct_answers = []

    # Generate the question stem using GPT-3.5-turbo
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a question generator."},
            {"role": "user", "content": f"Generate an MCQ question based on the following text: {input_text}"}
        ]
    )

    question = response['choices'][0]['message']['content'].strip()

    # Generate answer choices
    answer_choices = generate_answer_choices(input_text, num_choices)

    # Randomly select multiple correct answers
    correct_indices = random.sample(range(num_choices), num_correct_answers)
    correct_answers = [answer_choices[i] for i in correct_indices]

    return question, answer_choices, correct_answers

# Define a function to generate answer choices
def generate_answer_choices(input_text, num_choices):
    answer_choices = []

    # Here, you can generate answer choices based on the context and input text
    # For simplicity, we use placeholder choices
    for i in range(num_choices):
        answer_choices.append(f"Choice {chr(65 + i)}")

    return answer_choices

# Define a function to generate multiple MCQs
def generate_multiple_mcqs(input_text, num_mcqs, num_choices=4, num_correct_answers=2):
    mcqs = []

    for _ in range(num_mcqs):
        mcq_question, choices, correct_answers = generate_mcq_with_multiple_answers(input_text, num_choices, num_correct_answers)
        mcqs.append((mcq_question, choices, correct_answers))

    return mcqs

# Generate a list of MCQs
num_mcqs = 5  # Change this value to generate more MCQs
mcq_list = generate_multiple_mcqs(abstracted_text, num_mcqs)

# Print the generated MCQs and their answer choices
for i, (question, choices, correct_answers) in enumerate(mcq_list):
    print(f"MCQ Question {i + 1}: {question}")
    for j, choice in enumerate(choices):
        print(f"{chr(65 + j)}. {choice}")
    print(f"Correct Answers: {', '.join(correct_answers)}")
    print()


MCQ Question 1: Which battle marked the first major victory of the East India Company in India? 
a) Battle of Plassey 
b) Battle of Awadh 
c) Battle of Bengal 
d) Battle of Mysore
A. Choice A
B. Choice B
C. Choice C
D. Choice D
Correct Answers: Choice D, Choice A

MCQ Question 2: Which European power competed with the British East India Company in the Indian market during the rise of the company in India?

a) Portuguese
b) Dutch
c) French
d) All of the above
A. Choice A
B. Choice B
C. Choice C
D. Choice D
Correct Answers: Choice C, Choice D

MCQ Question 3: Which event led to the East India Company becoming the Diwan of the provinces of Bengal?

a) The Battle of Plassey
b) The decline of the Marathas
c) The 1857 revolt
d) The establishment of the Supreme Court
A. Choice A
B. Choice B
C. Choice C
D. Choice D
Correct Answers: Choice C, Choice A

MCQ Question 4: Question: What factors contributed to the rise of the British East India Company in India?

a) The decline of the Mughal empire 